<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Singapore Housing Data and Kaggle Challenge

_SG-DSI-41 Group 01: Daryl Chia, Germaine Choo, Lionel Foo_

---

## Part 3 Modeling

## 01. Import Libraries

In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# sklearn imports
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn import metrics

## 02. Model 1 - Non-Polynomial Features

### a. Load Data

In [2]:
# Load data:
model_1 = pd.read_csv('./output/model_1.csv')

# Display all columns:
pd.set_option('display.max_columns', None)

# Check data:
print(model_1.shape)
model_1.head()

(149805, 269)


,Unnamed: 0,hdb_age,max_floor_lvl,total_dwelling_units,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,mid_storey,floor_area_sqft,bus_stop_nearest_distance,mrt_nearest_distance,mall_nearest_distance,mall_within_500m,mall_within_1km,mall_within_2km,hawker_food_stalls,hawker_market_stalls,hawker_nearest_distance,hawker_within_500m,hawker_within_2km,vacancy,pri_sch_nearest_distance,cutoff_point,sec_sch_nearest_dist,1room_sold,commercial,market_hawker,multistorey_carpark,precinct_pavilion,bus_interchange,mrt_interchange,pri_sch_affiliation,affiliation,tranc_yearmonth_2012-04,tranc_yearmonth_2012-05,tranc_yearmonth_2012-06,tranc_yearmonth_2012-07,tranc_yearmonth_2012-08,tranc_yearmonth_2012-09,tranc_yearmonth_2012-10,tranc_yearmonth_2012-11,tranc_yearmonth_2012-12,tranc_yearmonth_2013-01,tranc_yearmonth_2013-02,tranc_yearmonth_2013-03,tranc_yearmonth_2013-04,tranc_yearmonth_2013-05,tranc_yearmonth_2013-06,tranc_yearmonth_2013-07,tranc_yearmonth_2013-08,tranc_yearmonth_2013-09,tranc_yearmonth_2013-10,tranc_yearmonth_2013-11,tranc_yearmonth_2013-12,tranc_yearmonth_2014-01,tranc_yearmonth_2014-02,tranc_yearmonth_2014-03,tranc_yearmonth_2014-04,tranc_yearmonth_2014-05,tranc_yearmonth_2014-06,tranc_yearmonth_2014-07,tranc_yearmonth_2014-08,tranc_yearmonth_2014-09,tranc_yearmonth_2014-10,tranc_yearmonth_2014-11,tranc_yearmonth_2014-12,tranc_yearmonth_2015-01,tranc_yearmonth_2015-02,tranc_yearmonth_2015-03,tranc_yearmonth_2015-04,tranc_yearmonth_2015-05,tranc_yearmonth_2015-06,tranc_yearmonth_2015-07,tranc_yearmonth_2015-08,tranc_yearmonth_2015-09,tranc_yearmonth_2015-10,tranc_yearmonth_2015-11,tranc_yearmonth_2015-12,tranc_yearmonth_2016-01,tranc_yearmonth_2016-02,tranc_yearmonth_2016-03,tranc_yearmonth_2016-04,tranc_yearmonth_2016-05,tranc_yearmonth_2016-06,tranc_yearmonth_2016-07,tranc_yearmonth_2016-08,tranc_yearmonth_2016-09,tranc_yearmonth_2016-10,tranc_yearmonth_2016-11,tranc_yearmonth_2016-12,tranc_yearmonth_2017-01,tranc_yearmonth_2017-02,tranc_yearmonth_2017-03,tranc_yearmonth_2017-04,tranc_yearmonth_2017-05,tranc_yearmonth_2017-06,tranc_yearmonth_2017-07,tranc_yearmonth_2017-08,tranc_yearmonth_2017-09,tranc_yearmonth_2017-10,tranc_yearmonth_2017-11,tranc_yearmonth_2017-12,tranc_yearmonth_2018-01,tranc_yearmonth_2018-02,tranc_yearmonth_2018-03,tranc_yearmonth_2018-04,tranc_yearmonth_2018-05,tranc_yearmonth_2018-06,tranc_yearmonth_2018-07,tranc_yearmonth_2018-08,tranc_yearmonth_2018-09,tranc_yearmonth_2018-10,tranc_yearmonth_2018-11,tranc_yearmonth_2018-12,tranc_yearmonth_2019-01,tranc_yearmonth_2019-02,tranc_yearmonth_2019-03,tranc_yearmonth_2019-04,tranc_yearmonth_2019-05,tranc_yearmonth_2019-06,tranc_yearmonth_2019-07,tranc_yearmonth_2019-08,tranc_yearmonth_2019-09,tranc_yearmonth_2019-10,tranc_yearmonth_2019-11,tranc_yearmonth_2019-12,tranc_yearmonth_2020-01,tranc_yearmonth_2020-02,tranc_yearmonth_2020-03,tranc_yearmonth_2020-04,tranc_yearmonth_2020-05,tranc_yearmonth_2020-06,tranc_yearmonth_2020-07,tranc_yearmonth_2020-08,tranc_yearmonth_2020-09,tranc_yearmonth_2020-10,tranc_yearmonth_2020-11,tranc_yearmonth_2020-12,tranc_yearmonth_2021-01,tranc_yearmonth_2021-02,tranc_yearmonth_2021-03,tranc_yearmonth_2021-04,planning_area_Bedok,planning_area_Bishan,planning_area_Bukit Batok,planning_area_Bukit Merah,planning_area_Bukit Panjang,planning_area_Bukit Timah,planning_area_Changi,planning_area_Choa Chu Kang,planning_area_Clementi,planning_area_Downtown Core,planning_area_Geylang,planning_area_Hougang,planning_area_Jurong East,planning_area_Jurong West,planning_area_Kallang,planning_area_Marine Parade,planning_area_Novena,planning_area_Outram,planning_area_Pasir Ris,planning_area_Punggol,planning_area_Queenstown,planning_area_Rochor,planning_area_Sembawang,planning_area_Sengkang,planning_area_Serangoon,planning_area_Tampines,planning_area_Tanglin,planning_area_Toa Payoh,planning_area_Western Water Catchment,planning_area_Woodlands,pl

In [3]:
# Drop the 'Unnamed: 0' column from Model_1
model_1.drop(columns = ['Unnamed: 0'], inplace = True)

### b. Separate Numerical and Categorical Features

In [4]:
# Identify binary categorical columns
model_1_binary_columns = [col for col in model_1.columns if model_1[col].nunique() == 2 and (model_1[col].dtype in ['int64', 'float64'] or model_1[col].dtype == 'bool')]

# Identify numerical columns
model_1_numerical_columns = [col for col in model_1.columns if col not in model_1_binary_columns and (model_1[col].dtype == 'int64' or model_1[col].dtype == 'float64')]

In [5]:
# Check binary columns 
model_1_binary_columns

['1room_sold',
 'commercial',
 'market_hawker',
 'multistorey_carpark',
 'precinct_pavilion',
 'bus_interchange',
 'mrt_interchange',
 'pri_sch_affiliation',
 'affiliation',
 'tranc_yearmonth_2012-04',
 'tranc_yearmonth_2012-05',
 'tranc_yearmonth_2012-06',
 'tranc_yearmonth_2012-07',
 'tranc_yearmonth_2012-08',
 'tranc_yearmonth_2012-09',
 'tranc_yearmonth_2012-10',
 'tranc_yearmonth_2012-11',
 'tranc_yearmonth_2012-12',
 'tranc_yearmonth_2013-01',
 'tranc_yearmonth_2013-02',
 'tranc_yearmonth_2013-03',
 'tranc_yearmonth_2013-04',
 'tranc_yearmonth_2013-05',
 'tranc_yearmonth_2013-06',
 'tranc_yearmonth_2013-07',
 'tranc_yearmonth_2013-08',
 'tranc_yearmonth_2013-09',
 'tranc_yearmonth_2013-10',
 'tranc_yearmonth_2013-11',
 'tranc_yearmonth_2013-12',
 'tranc_yearmonth_2014-01',
 'tranc_yearmonth_2014-02',
 'tranc_yearmonth_2014-03',
 'tranc_yearmonth_2014-04',
 'tranc_yearmonth_2014-05',
 'tranc_yearmonth_2014-06',
 'tranc_yearmonth_2014-07',
 'tranc_yearmonth_2014-08',
 'tranc_yearmo

In [6]:
# 1room_sold should be a numerical feature - shift 1room_sold from categorical features to numerical features
model_1_binary_columns.remove('1room_sold')

model_1_numerical_columns.append('1room_sold')

In [7]:
model_1_binary_columns

['commercial',
 'market_hawker',
 'multistorey_carpark',
 'precinct_pavilion',
 'bus_interchange',
 'mrt_interchange',
 'pri_sch_affiliation',
 'affiliation',
 'tranc_yearmonth_2012-04',
 'tranc_yearmonth_2012-05',
 'tranc_yearmonth_2012-06',
 'tranc_yearmonth_2012-07',
 'tranc_yearmonth_2012-08',
 'tranc_yearmonth_2012-09',
 'tranc_yearmonth_2012-10',
 'tranc_yearmonth_2012-11',
 'tranc_yearmonth_2012-12',
 'tranc_yearmonth_2013-01',
 'tranc_yearmonth_2013-02',
 'tranc_yearmonth_2013-03',
 'tranc_yearmonth_2013-04',
 'tranc_yearmonth_2013-05',
 'tranc_yearmonth_2013-06',
 'tranc_yearmonth_2013-07',
 'tranc_yearmonth_2013-08',
 'tranc_yearmonth_2013-09',
 'tranc_yearmonth_2013-10',
 'tranc_yearmonth_2013-11',
 'tranc_yearmonth_2013-12',
 'tranc_yearmonth_2014-01',
 'tranc_yearmonth_2014-02',
 'tranc_yearmonth_2014-03',
 'tranc_yearmonth_2014-04',
 'tranc_yearmonth_2014-05',
 'tranc_yearmonth_2014-06',
 'tranc_yearmonth_2014-07',
 'tranc_yearmonth_2014-08',
 'tranc_yearmonth_2014-09',
 

In [8]:
model_1_numerical_columns

['hdb_age',
 'max_floor_lvl',
 'total_dwelling_units',
 '2room_sold',
 '3room_sold',
 '4room_sold',
 '5room_sold',
 'exec_sold',
 'multigen_sold',
 'studio_apartment_sold',
 '1room_rental',
 '2room_rental',
 '3room_rental',
 'other_room_rental',
 'mid_storey',
 'floor_area_sqft',
 'bus_stop_nearest_distance',
 'mrt_nearest_distance',
 'mall_nearest_distance',
 'mall_within_500m',
 'mall_within_1km',
 'mall_within_2km',
 'hawker_food_stalls',
 'hawker_market_stalls',
 'hawker_nearest_distance',
 'hawker_within_500m',
 'hawker_within_2km',
 'vacancy',
 'pri_sch_nearest_distance',
 'cutoff_point',
 'sec_sch_nearest_dist',
 'resale_price',
 '1room_sold']

In [9]:
# Drop target variable 'resale_price'from numerical columns
model_1_numerical_columns.remove('resale_price')

# check numerical columns
model_1_numerical_columns

['hdb_age',
 'max_floor_lvl',
 'total_dwelling_units',
 '2room_sold',
 '3room_sold',
 '4room_sold',
 '5room_sold',
 'exec_sold',
 'multigen_sold',
 'studio_apartment_sold',
 '1room_rental',
 '2room_rental',
 '3room_rental',
 'other_room_rental',
 'mid_storey',
 'floor_area_sqft',
 'bus_stop_nearest_distance',
 'mrt_nearest_distance',
 'mall_nearest_distance',
 'mall_within_500m',
 'mall_within_1km',
 'mall_within_2km',
 'hawker_food_stalls',
 'hawker_market_stalls',
 'hawker_nearest_distance',
 'hawker_within_500m',
 'hawker_within_2km',
 'vacancy',
 'pri_sch_nearest_distance',
 'cutoff_point',
 'sec_sch_nearest_dist',
 '1room_sold']

In [10]:
# Check total count of columns
len(model_1_binary_columns + model_1_numerical_columns)

267

In [11]:
# Separate binary and numerical columns
model_1_binary_data = model_1[model_1_binary_columns]
model_1_numerical_data = model_1[model_1_numerical_columns]

### c. Scale numerical data

In [12]:
# Standardize the numerical features using StandardScaler
ss = StandardScaler()
model_1_numerical_data_scaled = ss.fit_transform(model_1_numerical_data)

### d. Creating Features (X) and Target Variable (y)

In [13]:
# Concatenate scaled numerical columns with binary columns
model_1_combined_data = pd.concat([pd.DataFrame(model_1_numerical_data_scaled, columns=model_1_numerical_columns), model_1_binary_data], axis=1)

In [14]:
# Separate the target variable 'resale_price' (y) and features (X) from model_1
y = model_1['resale_price']
X = model_1_combined_data

In [15]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### e. Model Development

In [16]:
# Instantiate a Linear Regression model
lr = LinearRegression()

In [17]:
# Instantiate a Lasso Regression model
lasso = LassoCV(n_alphas = 200, max_iter=100000)

In [18]:
# Instantiate a Ridge Regression model
r_alphas = np.logspace(-2, 2, 100)
ridge = RidgeCV(alphas = r_alphas)

In [19]:
# Calculate cross-validated scores for Linear Regression
lr_scores = cross_val_score(lr, X_train, y_train, cv=5)

# Printing the scores
print("Cross-validated R2 scores for Linear Regression model:")
print(lr_scores)
print("Mean R2 score:", np.mean(lr_scores))

Cross-validated R2 scores for Linear Regression model:
[ 9.02030629e-01  9.01475462e-01  9.04573313e-01  8.98199469e-01
 -3.26889703e+19]
Mean R2 score: -6.537794064580346e+18


In [20]:
# Cross-validation for Lasso Regression
lasso_scores = cross_val_score(lasso, X_train, y_train, cv=5)

# Printing the scores
print("Cross-validated R2 scores for Lasso Regression model:")
print(lasso_scores)
print("Mean R2 score:", np.mean(lasso_scores))

Cross-validated R2 scores for Lasso Regression model:
[0.89675987 0.89565799 0.89894004 0.89202813 0.89656935]
Mean R2 score: 0.8959910742072242


In [21]:
# Cross-validation for Ridge Regression
ridge_scores = cross_val_score(ridge, X_train, y_train, cv=5)

# Printing the scores
print("Cross-validated R2 scores for Ridge Regression model:")
print(ridge_scores)
print("Mean R2 score:", np.mean(ridge_scores))

Cross-validated R2 scores for Ridge Regression model:
[0.90206307 0.90148898 0.90466009 0.89819626 0.90389957]
Mean R2 score: 0.9020615953522952


In [22]:
# Fit the Lasso model to the training data
lasso.fit(X_train, y_train)

LassoCV(max_iter=100000, n_alphas=200)

In [23]:
# Calculate R2 scores
train_r2_lasso = lasso.score(X_train, y_train)
test_r2_lasso = lasso.score(X_test, y_test)

# Calculate RMSE
train_rmse_lasso = np.sqrt(metrics.mean_squared_error(y_train, lasso.predict(X_train)))
test_rmse_lasso = np.sqrt(metrics.mean_squared_error(y_test, lasso.predict(X_test)))

# Print the results
print("Lasso Model - Training R2 score:", train_r2_lasso)
print("Lasso Model - Testing R2 score:", test_r2_lasso)
print("Lasso Model - Training RMSE:", train_rmse_lasso)
print("Lasso Model - Testing RMSE:", test_rmse_lasso)

Lasso Model - Training R2 score: 0.8962649559522281
Lasso Model - Testing R2 score: 0.8932240440099044
Lasso Model - Training RMSE: 46142.237746969535
Lasso Model - Testing RMSE: 46728.646492120795


In [24]:
# Fit the Ridge model to the training data
ridge.fit(X_train, y_train)

RidgeCV(alphas=array([1.00000000e-02, 1.09749877e-02, 1.20450354e-02, 1.32194115e-02,
       1.45082878e-02, 1.59228279e-02, 1.74752840e-02, 1.91791026e-02,
       2.10490414e-02, 2.31012970e-02, 2.53536449e-02, 2.78255940e-02,
       3.05385551e-02, 3.35160265e-02, 3.67837977e-02, 4.03701726e-02,
       4.43062146e-02, 4.86260158e-02, 5.33669923e-02, 5.85702082e-02,
       6.42807312e-02, 7.05480231e-0...
       1.17681195e+01, 1.29154967e+01, 1.41747416e+01, 1.55567614e+01,
       1.70735265e+01, 1.87381742e+01, 2.05651231e+01, 2.25701972e+01,
       2.47707636e+01, 2.71858824e+01, 2.98364724e+01, 3.27454916e+01,
       3.59381366e+01, 3.94420606e+01, 4.32876128e+01, 4.75081016e+01,
       5.21400829e+01, 5.72236766e+01, 6.28029144e+01, 6.89261210e+01,
       7.56463328e+01, 8.30217568e+01, 9.11162756e+01, 1.00000000e+02]))

In [25]:
# Calculate R2 scores
train_r2_ridge = ridge.score(X_train, y_train)
test_r2_ridge = ridge.score(X_test, y_test)

# Calculate RMSE
train_rmse_ridge = np.sqrt(metrics.mean_squared_error(y_train, ridge.predict(X_train)))
test_rmse_ridge = np.sqrt(metrics.mean_squared_error(y_test, ridge.predict(X_test)))

# Print the results
print("Ridge Model - Training R2 score:", train_r2_ridge)
print("Ridge Model - Testing R2 score:", test_r2_ridge)
print("Ridge Model - Training RMSE:", train_rmse_ridge)
print("Ridge Model - Testing RMSE:", test_rmse_ridge)

Ridge Model - Training R2 score: 0.902553977762096
Ridge Model - Testing R2 score: 0.9003599474289852
Ridge Model - Training RMSE: 44721.66485026595
Ridge Model - Testing RMSE: 45140.19610994031


## 03. Model 2 - Polynomial Features (Unadjusted Model)

### a. Load Data

In [33]:
# Load data:
model_2 = pd.read_csv('./output/model_2.csv')

# Check data:
print(model_2.shape)
model_2.head()

(149805, 797)


,Unnamed: 0,hdb_age,max_floor_lvl,total_dwelling_units,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,mid_storey,floor_area_sqft,bus_stop_nearest_distance,mrt_nearest_distance,mall_nearest_distance,mall_within_500m,mall_within_1km,mall_within_2km,hawker_food_stalls,hawker_market_stalls,hawker_nearest_distance,hawker_within_500m,hawker_within_2km,vacancy,pri_sch_nearest_distance,cutoff_point,sec_sch_nearest_dist,1room_sold,hdb_age^2,hdb_age max_floor_lvl,hdb_age total_dwelling_units,hdb_age 2room_sold,hdb_age 3room_sold,hdb_age 4room_sold,hdb_age 5room_sold,hdb_age exec_sold,hdb_age multigen_sold,hdb_age studio_apartment_sold,hdb_age 1room_rental,hdb_age 2room_rental,hdb_age 3room_rental,hdb_age other_room_rental,hdb_age mid_storey,hdb_age floor_area_sqft,hdb_age bus_stop_nearest_distance,hdb_age mrt_nearest_distance,hdb_age mall_nearest_distance,hdb_age mall_within_500m,hdb_age mall_within_1km,hdb_age mall_within_2km,hdb_age hawker_food_stalls,hdb_age hawker_market_stalls,hdb_age hawker_nearest_distance,hdb_age hawker_within_500m,hdb_age hawker_within_2km,hdb_age vacancy,hdb_age pri_sch_nearest_distance,hdb_age cutoff_point,hdb_age sec_sch_nearest_dist,hdb_age 1room_sold,max_floor_lvl^2,max_floor_lvl total_dwelling_units,max_floor_lvl 2room_sold,max_floor_lvl 3room_sold,max_floor_lvl 4room_sold,max_floor_lvl 5room_sold,max_floor_lvl exec_sold,max_floor_lvl multigen_sold,max_floor_lvl studio_apartment_sold,max_floor_lvl 1room_rental,max_floor_lvl 2room_rental,max_floor_lvl 3room_rental,max_floor_lvl other_room_rental,max_floor_lvl mid_storey,max_floor_lvl floor_area_sqft,max_floor_lvl bus_stop_nearest_distance,max_floor_lvl mrt_nearest_distance,max_floor_lvl mall_nearest_distance,max_floor_lvl mall_within_500m,max_floor_lvl mall_within_1km,max_floor_lvl mall_within_2km,max_floor_lvl hawker_food_stalls,max_floor_lvl hawker_market_stalls,max_floor_lvl hawker_nearest_distance,max_floor_lvl hawker_within_500m,max_floor_lvl hawker_within_2km,max_floor_lvl vacancy,max_floor_lvl pri_sch_nearest_distance,max_floor_lvl cutoff_point,max_floor_lvl sec_sch_nearest_dist,max_floor_lvl 1room_sold,total_dwelling_units^2,total_dwelling_units 2room_sold,total_dwelling_units 3room_sold,total_dwelling_units 4room_sold,total_dwelling_units 5room_sold,total_dwelling_units exec_sold,total_dwelling_units multigen_sold,total_dwelling_units studio_apartment_sold,total_dwelling_units 1room_rental,total_dwelling_units 2room_rental,total_dwelling_units 3room_rental,total_dwelling_units other_room_rental,total_dwelling_units mid_storey,total_dwelling_units floor_area_sqft,total_dwelling_units bus_stop_nearest_distance,total_dwelling_units mrt_nearest_distance,total_dwelling_units mall_nearest_distance,total_dwelling_units mall_within_500m,total_dwelling_units mall_within_1km,total_dwelling_units mall_within_2km,total_dwelling_units hawker_food_stalls,total_dwelling_units hawker_market_stalls,total_dwelling_units hawker_nearest_distance,total_dwelling_units hawker_within_500m,total_dwelling_units hawker_within_2km,total_dwelling_units vacancy,total_dwelling_units pri_sch_nearest_distance,total_dwelling_units cutoff_point,total_dwelling_units sec_sch_nearest_dist,total_dwelling_units 1room_sold,2room_sold^2,2room_sold 3room_sold,2room_sold 4room_sold,2room_sold 5room_sold,2room_sold exec_sold,2room_sold multigen_sold,2room_sold studio_apartment_sold,2room_sold 1room_rental,2room_sold 2room_rental,2room_sold 3room_rental,2room_sold other_room_rental,2room_sold mid_storey,2room_sold floor_area_sqft,2room_sold bus_stop_nearest_distance,2room_sold mrt_nearest_distance,2room_sold mall_nearest_distance,2room_sold mall_within_500m,2room_sold mall_within_1km,2room_sold mall_within_2km,2room_sold hawker_food_stalls,2room_sold hawker_market_stalls,2room_sold hawker_nearest_distance,2room_sold hawker_within_500m,2room_sold hawker_within_2km,2room_sold vacancy,2room_sold p

In [34]:
# Drop the 'Unnamed: 0' column from Model_2
model_2.drop(columns = ['Unnamed: 0'], inplace = True)

### b. Separate Numerical and Categorical Features

In [35]:
# Identify binary categorical columns (binary categorical columns have no changes across all our models)
model_2_binary_columns = model_1_binary_columns.copy()

model_2_binary_columns

['commercial',
 'market_hawker',
 'multistorey_carpark',
 'precinct_pavilion',
 'bus_interchange',
 'mrt_interchange',
 'pri_sch_affiliation',
 'affiliation',
 'tranc_yearmonth_2012-04',
 'tranc_yearmonth_2012-05',
 'tranc_yearmonth_2012-06',
 'tranc_yearmonth_2012-07',
 'tranc_yearmonth_2012-08',
 'tranc_yearmonth_2012-09',
 'tranc_yearmonth_2012-10',
 'tranc_yearmonth_2012-11',
 'tranc_yearmonth_2012-12',
 'tranc_yearmonth_2013-01',
 'tranc_yearmonth_2013-02',
 'tranc_yearmonth_2013-03',
 'tranc_yearmonth_2013-04',
 'tranc_yearmonth_2013-05',
 'tranc_yearmonth_2013-06',
 'tranc_yearmonth_2013-07',
 'tranc_yearmonth_2013-08',
 'tranc_yearmonth_2013-09',
 'tranc_yearmonth_2013-10',
 'tranc_yearmonth_2013-11',
 'tranc_yearmonth_2013-12',
 'tranc_yearmonth_2014-01',
 'tranc_yearmonth_2014-02',
 'tranc_yearmonth_2014-03',
 'tranc_yearmonth_2014-04',
 'tranc_yearmonth_2014-05',
 'tranc_yearmonth_2014-06',
 'tranc_yearmonth_2014-07',
 'tranc_yearmonth_2014-08',
 'tranc_yearmonth_2014-09',
 

In [36]:
# Identify numerical columns
model_2_numerical_columns = model_2.columns.copy()
model_2_numerical_columns = [i for i in model_2_numerical_columns if i not in model_2_binary_columns]

In [37]:
# Check binary columns 
model_2_binary_columns

['commercial',
 'market_hawker',
 'multistorey_carpark',
 'precinct_pavilion',
 'bus_interchange',
 'mrt_interchange',
 'pri_sch_affiliation',
 'affiliation',
 'tranc_yearmonth_2012-04',
 'tranc_yearmonth_2012-05',
 'tranc_yearmonth_2012-06',
 'tranc_yearmonth_2012-07',
 'tranc_yearmonth_2012-08',
 'tranc_yearmonth_2012-09',
 'tranc_yearmonth_2012-10',
 'tranc_yearmonth_2012-11',
 'tranc_yearmonth_2012-12',
 'tranc_yearmonth_2013-01',
 'tranc_yearmonth_2013-02',
 'tranc_yearmonth_2013-03',
 'tranc_yearmonth_2013-04',
 'tranc_yearmonth_2013-05',
 'tranc_yearmonth_2013-06',
 'tranc_yearmonth_2013-07',
 'tranc_yearmonth_2013-08',
 'tranc_yearmonth_2013-09',
 'tranc_yearmonth_2013-10',
 'tranc_yearmonth_2013-11',
 'tranc_yearmonth_2013-12',
 'tranc_yearmonth_2014-01',
 'tranc_yearmonth_2014-02',
 'tranc_yearmonth_2014-03',
 'tranc_yearmonth_2014-04',
 'tranc_yearmonth_2014-05',
 'tranc_yearmonth_2014-06',
 'tranc_yearmonth_2014-07',
 'tranc_yearmonth_2014-08',
 'tranc_yearmonth_2014-09',
 

In [39]:
# Check numerical columns
model_2_numerical_columns

['hdb_age',
 'max_floor_lvl',
 'total_dwelling_units',
 '2room_sold',
 '3room_sold',
 '4room_sold',
 '5room_sold',
 'exec_sold',
 'multigen_sold',
 'studio_apartment_sold',
 '1room_rental',
 '2room_rental',
 '3room_rental',
 'other_room_rental',
 'mid_storey',
 'floor_area_sqft',
 'bus_stop_nearest_distance',
 'mrt_nearest_distance',
 'mall_nearest_distance',
 'mall_within_500m',
 'mall_within_1km',
 'mall_within_2km',
 'hawker_food_stalls',
 'hawker_market_stalls',
 'hawker_nearest_distance',
 'hawker_within_500m',
 'hawker_within_2km',
 'vacancy',
 'pri_sch_nearest_distance',
 'cutoff_point',
 'sec_sch_nearest_dist',
 '1room_sold',
 'hdb_age^2',
 'hdb_age max_floor_lvl',
 'hdb_age total_dwelling_units',
 'hdb_age 2room_sold',
 'hdb_age 3room_sold',
 'hdb_age 4room_sold',
 'hdb_age 5room_sold',
 'hdb_age exec_sold',
 'hdb_age multigen_sold',
 'hdb_age studio_apartment_sold',
 'hdb_age 1room_rental',
 'hdb_age 2room_rental',
 'hdb_age 3room_rental',
 'hdb_age other_room_rental',
 'hdb_

In [40]:
len(model_2_numerical_columns) + len(model_2_binary_columns)

796

In [41]:
# Drop target variable 'resale_price'from numerical columns
model_2_numerical_columns.remove('resale_price')

# check numerical columns
model_2_numerical_columns

['hdb_age',
 'max_floor_lvl',
 'total_dwelling_units',
 '2room_sold',
 '3room_sold',
 '4room_sold',
 '5room_sold',
 'exec_sold',
 'multigen_sold',
 'studio_apartment_sold',
 '1room_rental',
 '2room_rental',
 '3room_rental',
 'other_room_rental',
 'mid_storey',
 'floor_area_sqft',
 'bus_stop_nearest_distance',
 'mrt_nearest_distance',
 'mall_nearest_distance',
 'mall_within_500m',
 'mall_within_1km',
 'mall_within_2km',
 'hawker_food_stalls',
 'hawker_market_stalls',
 'hawker_nearest_distance',
 'hawker_within_500m',
 'hawker_within_2km',
 'vacancy',
 'pri_sch_nearest_distance',
 'cutoff_point',
 'sec_sch_nearest_dist',
 '1room_sold',
 'hdb_age^2',
 'hdb_age max_floor_lvl',
 'hdb_age total_dwelling_units',
 'hdb_age 2room_sold',
 'hdb_age 3room_sold',
 'hdb_age 4room_sold',
 'hdb_age 5room_sold',
 'hdb_age exec_sold',
 'hdb_age multigen_sold',
 'hdb_age studio_apartment_sold',
 'hdb_age 1room_rental',
 'hdb_age 2room_rental',
 'hdb_age 3room_rental',
 'hdb_age other_room_rental',
 'hdb_

In [42]:
# Check total count of columns
len(model_2_binary_columns + model_2_numerical_columns)

795

In [43]:
# Separate binary and numerical columns
model_2_binary_data = model_2[model_2_binary_columns]
model_2_numerical_data = model_2[model_2_numerical_columns]

### c. Scale numerical data

In [44]:
# Standardize the numerical features using StandardScaler
ss2 = StandardScaler()
model_2_numerical_data_scaled = ss2.fit_transform(model_2_numerical_data)

### d. Creating Features (X) and Target Variable (y)

In [45]:
# Concatenate scaled numerical columns with binary columns
model_2_combined_data = pd.concat([pd.DataFrame(model_2_numerical_data_scaled, columns=model_2_numerical_columns), model_2_binary_data], axis=1)

In [46]:
# Separate the target variable 'resale_price' (y) and features (X) from model_3
y2 = model_2['resale_price']
X2 = model_2_combined_data

In [47]:
# Split the dataset into training and testing sets
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

### e. Model Development

In [48]:
# Instantiate a Linear Regression model
lr2 = LinearRegression()

In [49]:
# Instantiate a Lasso Regression model
lasso2 = LassoCV(n_alphas = 200, max_iter=100000)

In [50]:
# Instantiate a Ridge Regression model
r_alphas = np.logspace(-2, 2, 100)
ridge2 = RidgeCV(alphas = r_alphas)

In [51]:
# Calculate cross-validated scores for Linear Regression
lr_scores2 = cross_val_score(lr2, X_train2, y_train2, cv=5)

# Printing the scores
print("Cross-validated R2 scores for Linear Regression model:")
print(lr_scores2)
print("Mean R2 score:", np.mean(lr_scores2))

Cross-validated R2 scores for Linear Regression model:
[ 9.31911856e-01  9.32167258e-01 -2.62390832e+18  9.29611511e-01
 -1.81994467e+13]
Mean R2 score: -5.247853037749262e+17


In [52]:
# Cross-validation for Lasso Regression
lasso_scores2 = cross_val_score(lasso2, X_train2, y_train2, cv=5)

# Printing the scores
print("Cross-validated R2 scores for Lasso Regression model:")
print(lasso_scores2)
print("Mean R2 score:", np.mean(lasso_scores2))

Cross-validated R2 scores for Lasso Regression model:
[0.922627   0.9222594  0.92288786 0.91836733 0.92149309]
Mean R2 score: 0.9215269359802472


In [53]:
# Cross-validation for Ridge Regression
ridge_scores2 = cross_val_score(ridge2, X_train2, y_train2, cv=5)

# Printing the scores
print("Cross-validated R2 scores for Ridge Regression model:")
print(ridge_scores2)
print("Mean R2 score:", np.mean(ridge_scores2))

Cross-validated R2 scores for Ridge Regression model:
[0.93192775 0.93216163 0.9330475  0.92968029 0.93270115]
Mean R2 score: 0.9319036647321136


In [54]:
# Fit the Lasso model to the training data
lasso2.fit(X_train2, y_train2)

LassoCV(max_iter=100000, n_alphas=200)

In [55]:
# Calculate R2 scores
train_r2_lasso2 = lasso2.score(X_train2, y_train2)
test_r2_lasso2 = lasso2.score(X_test2, y_test2)

# Calculate RMSE
train_rmse_lasso2 = np.sqrt(metrics.mean_squared_error(y_train2, lasso2.predict(X_train2)))
test_rmse_lasso2 = np.sqrt(metrics.mean_squared_error(y_test2, lasso2.predict(X_test2)))

# Print the results
print("Lasso Model - Training R2 score:", train_r2_lasso2)
print("Lasso Model - Testing R2 score:", test_r2_lasso2)
print("Lasso Model - Training RMSE:", train_rmse_lasso2)
print("Lasso Model - Testing RMSE:", test_rmse_lasso2)

Lasso Model - Training R2 score: 0.9221476450920951
Lasso Model - Testing R2 score: 0.9202347958515381
Lasso Model - Training RMSE: 39973.46371015888
Lasso Model - Testing RMSE: 40388.08002003619


In [56]:
# Fit the Ridge model to the training data
ridge2.fit(X_train2, y_train2)

RidgeCV(alphas=array([1.00000000e-02, 1.09749877e-02, 1.20450354e-02, 1.32194115e-02,
       1.45082878e-02, 1.59228279e-02, 1.74752840e-02, 1.91791026e-02,
       2.10490414e-02, 2.31012970e-02, 2.53536449e-02, 2.78255940e-02,
       3.05385551e-02, 3.35160265e-02, 3.67837977e-02, 4.03701726e-02,
       4.43062146e-02, 4.86260158e-02, 5.33669923e-02, 5.85702082e-02,
       6.42807312e-02, 7.05480231e-0...
       1.17681195e+01, 1.29154967e+01, 1.41747416e+01, 1.55567614e+01,
       1.70735265e+01, 1.87381742e+01, 2.05651231e+01, 2.25701972e+01,
       2.47707636e+01, 2.71858824e+01, 2.98364724e+01, 3.27454916e+01,
       3.59381366e+01, 3.94420606e+01, 4.32876128e+01, 4.75081016e+01,
       5.21400829e+01, 5.72236766e+01, 6.28029144e+01, 6.89261210e+01,
       7.56463328e+01, 8.30217568e+01, 9.11162756e+01, 1.00000000e+02]))

In [57]:
# Calculate R2 scores
train_r2_ridge2 = ridge2.score(X_train2, y_train2)
test_r2_ridge2 = ridge2.score(X_test2, y_test2)

# Calculate RMSE
train_rmse_ridge2 = np.sqrt(metrics.mean_squared_error(y_train2, ridge2.predict(X_train2)))
test_rmse_ridge2 = np.sqrt(metrics.mean_squared_error(y_test2, ridge2.predict(X_test2)))

# Print the results
print("Ridge Model - Training R2 score:", train_r2_ridge2)
print("Ridge Model - Testing R2 score:", test_r2_ridge2)
print("Ridge Model - Training RMSE:", train_rmse_ridge2)
print("Ridge Model - Testing RMSE:", test_rmse_ridge2)

Ridge Model - Training R2 score: 0.9329776804491743
Ridge Model - Testing R2 score: 0.9319617934263061
Ridge Model - Training RMSE: 37089.04336569738
Ridge Model - Testing RMSE: 37301.20796497079


In [58]:
coefficients = ridge2.coef_

# Create a new DataFrame with predictors and coefficients
df_ridge = pd.DataFrame({"predictors": X2.columns, "coefficients": coefficients})

# Sort the DataFrame ordered from largest to smallest coefficient by absolute values
df_ridge_sorted = df_ridge.loc[df_ridge["coefficients"].abs().sort_values(ascending=False).index]

# Reset the index
df_ridge_sorted.reset_index(drop=True, inplace=True)

# Display all rows:
pd.set_option('display.max_rows', None)

#sorted dataframes
print("\nModel 2 Coefficients:")
print(df_ridge_sorted)


Model 2 Coefficients:
                                            predictors   coefficients
0                                planning_area_Tanglin  212969.487723
1                                   flat_model_Terrace  208520.784576
2                            planning_area_Bukit Timah  155842.917900
3                                 planning_area_Changi -154106.757788
4                        full_flat_type_3 ROOM Terrace  140623.880019
5                                        market_hawker -137534.450304
6                              planning_area_Sembawang -131909.504632
7                          planning_area_Marine Parade  127589.261734
8                                         cutoff_point -122048.898077
9                              planning_area_Woodlands -108623.068477
10                                      cutoff_point^2  101126.287922
11                                planning_area_Outram   98661.851436
12                         planning_area_Choa Chu Kang  -90691.8861

## 03. Model 3 - Polynomial Features  (Adjusted for Multicollinearity)

### a. Load Data

In [59]:
# Load data:
model_3 = pd.read_csv('./output/model_3.csv')

# Check data:
print(model_3.shape)
model_3.head()

(149805, 504)


,Unnamed: 0,2room_sold,3room_sold,floor_area_sqft,hawker_food_stalls,vacancy,hdb_age^2,hdb_age max_floor_lvl,hdb_age 2room_sold,hdb_age 4room_sold,hdb_age 1room_rental,hdb_age mid_storey,hdb_age floor_area_sqft,hdb_age bus_stop_nearest_distance,hdb_age mrt_nearest_distance,hdb_age mall_nearest_distance,hdb_age mall_within_1km,hdb_age mall_within_2km,hdb_age hawker_food_stalls,hdb_age hawker_market_stalls,hdb_age hawker_nearest_distance,hdb_age hawker_within_500m,hdb_age vacancy,hdb_age pri_sch_nearest_distance,hdb_age sec_sch_nearest_dist,hdb_age 1room_sold,max_floor_lvl 4room_sold,max_floor_lvl 5room_sold,max_floor_lvl floor_area_sqft,max_floor_lvl bus_stop_nearest_distance,max_floor_lvl mall_nearest_distance,max_floor_lvl mall_within_500m,max_floor_lvl mall_within_1km,max_floor_lvl mall_within_2km,max_floor_lvl hawker_food_stalls,max_floor_lvl hawker_market_stalls,max_floor_lvl hawker_within_500m,max_floor_lvl hawker_within_2km,max_floor_lvl vacancy,max_floor_lvl pri_sch_nearest_distance,max_floor_lvl sec_sch_nearest_dist,total_dwelling_units floor_area_sqft,total_dwelling_units bus_stop_nearest_distance,total_dwelling_units mrt_nearest_distance,total_dwelling_units mall_nearest_distance,total_dwelling_units mall_within_2km,total_dwelling_units hawker_food_stalls,total_dwelling_units hawker_nearest_distance,total_dwelling_units pri_sch_nearest_distance,total_dwelling_units cutoff_point,total_dwelling_units sec_sch_nearest_dist,2room_sold 3room_sold,2room_sold 4room_sold,2room_sold 5room_sold,2room_sold exec_sold,2room_sold multigen_sold,2room_sold 2room_rental,2room_sold 3room_rental,2room_sold other_room_rental,2room_sold mall_within_500m,2room_sold mall_within_2km,2room_sold hawker_market_stalls,2room_sold hawker_nearest_distance,2room_sold 1room_sold,3room_sold 4room_sold,3room_sold 5room_sold,3room_sold exec_sold,3room_sold multigen_sold,3room_sold studio_apartment_sold,3room_sold 1room_rental,3room_sold other_room_rental,3room_sold mall_within_500m,3room_sold mall_within_1km,3room_sold mall_within_2km,3room_sold hawker_nearest_distance,3room_sold hawker_within_500m,4room_sold 5room_sold,4room_sold exec_sold,4room_sold studio_apartment_sold,4room_sold 2room_rental,4room_sold 3room_rental,4room_sold bus_stop_nearest_distance,4room_sold mrt_nearest_distance,4room_sold mall_nearest_distance,4room_sold mall_within_500m,4room_sold mall_within_1km,4room_sold mall_within_2km,4room_sold hawker_food_stalls,4room_sold hawker_market_stalls,4room_sold hawker_nearest_distance,4room_sold hawker_within_500m,4room_sold hawker_within_2km,4room_sold vacancy,4room_sold pri_sch_nearest_distance,4room_sold sec_sch_nearest_dist,4room_sold 1room_sold,5room_sold exec_sold,5room_sold multigen_sold,5room_sold studio_apartment_sold,5room_sold 1room_rental,5room_sold 2room_rental,5room_sold 3room_rental,5room_sold other_room_rental,5room_sold bus_stop_nearest_distance,5room_sold mrt_nearest_distance,5room_sold mall_nearest_distance,5room_sold mall_within_500m,5room_sold mall_within_1km,5room_sold hawker_food_stalls,5room_sold hawker_market_stalls,5room_sold hawker_nearest_distance,5room_sold hawker_within_500m,5room_sold hawker_within_2km,5room_sold pri_sch_nearest_distance,5room_sold sec_sch_nearest_dist,5room_sold 1room_sold,exec_sold multigen_sold,exec_sold studio_apartment_sold,exec_sold 1room_rental,exec_sold 2room_rental,exec_sold 3room_rental,exec_sold other_room_rental,exec_sold floor_area_sqft,exec_sold mall_within_500m,exec_sold mall_within_1km,exec_sold hawker_market_stalls,exec_sold hawker_nearest_distance,exec_sold hawker_within_500m,exec_sold hawker_within_2km,exec_sold 1room_sold,multigen_sold studio_apartment_sold,multigen_sold 1room_rental,multigen_sold 2room_rental,multigen_sold 3room_rental,multigen_sold other_room_rental,multigen_sold floor_area_sqft,multigen_sold mall_nearest_distance,multigen_sold hawker_market_stalls,multigen_sold 1room_sold,studio_apartment_sold 1room_rental,studio_apartment_sold 2room_rental,studio_apartmen

In [60]:
# Drop the 'Unnamed: 0' column from Model_3
model_3.drop(columns = ['Unnamed: 0'], inplace = True)


### b. Separate Numerical and Categorical Features

In [61]:
# Identify binary categorical columns (binary categorical columns have no changes across all our models)
model_3_binary_columns = model_1_binary_columns.copy()

model_3_binary_columns

['commercial',
 'market_hawker',
 'multistorey_carpark',
 'precinct_pavilion',
 'bus_interchange',
 'mrt_interchange',
 'pri_sch_affiliation',
 'affiliation',
 'tranc_yearmonth_2012-04',
 'tranc_yearmonth_2012-05',
 'tranc_yearmonth_2012-06',
 'tranc_yearmonth_2012-07',
 'tranc_yearmonth_2012-08',
 'tranc_yearmonth_2012-09',
 'tranc_yearmonth_2012-10',
 'tranc_yearmonth_2012-11',
 'tranc_yearmonth_2012-12',
 'tranc_yearmonth_2013-01',
 'tranc_yearmonth_2013-02',
 'tranc_yearmonth_2013-03',
 'tranc_yearmonth_2013-04',
 'tranc_yearmonth_2013-05',
 'tranc_yearmonth_2013-06',
 'tranc_yearmonth_2013-07',
 'tranc_yearmonth_2013-08',
 'tranc_yearmonth_2013-09',
 'tranc_yearmonth_2013-10',
 'tranc_yearmonth_2013-11',
 'tranc_yearmonth_2013-12',
 'tranc_yearmonth_2014-01',
 'tranc_yearmonth_2014-02',
 'tranc_yearmonth_2014-03',
 'tranc_yearmonth_2014-04',
 'tranc_yearmonth_2014-05',
 'tranc_yearmonth_2014-06',
 'tranc_yearmonth_2014-07',
 'tranc_yearmonth_2014-08',
 'tranc_yearmonth_2014-09',
 

In [62]:
# Identify numerical columns
model_3_numerical_columns = model_3.columns.copy()
model_3_numerical_columns = [i for i in model_3_numerical_columns if i not in model_3_binary_columns]

In [63]:
# Check binary columns 
model_3_binary_columns

['commercial',
 'market_hawker',
 'multistorey_carpark',
 'precinct_pavilion',
 'bus_interchange',
 'mrt_interchange',
 'pri_sch_affiliation',
 'affiliation',
 'tranc_yearmonth_2012-04',
 'tranc_yearmonth_2012-05',
 'tranc_yearmonth_2012-06',
 'tranc_yearmonth_2012-07',
 'tranc_yearmonth_2012-08',
 'tranc_yearmonth_2012-09',
 'tranc_yearmonth_2012-10',
 'tranc_yearmonth_2012-11',
 'tranc_yearmonth_2012-12',
 'tranc_yearmonth_2013-01',
 'tranc_yearmonth_2013-02',
 'tranc_yearmonth_2013-03',
 'tranc_yearmonth_2013-04',
 'tranc_yearmonth_2013-05',
 'tranc_yearmonth_2013-06',
 'tranc_yearmonth_2013-07',
 'tranc_yearmonth_2013-08',
 'tranc_yearmonth_2013-09',
 'tranc_yearmonth_2013-10',
 'tranc_yearmonth_2013-11',
 'tranc_yearmonth_2013-12',
 'tranc_yearmonth_2014-01',
 'tranc_yearmonth_2014-02',
 'tranc_yearmonth_2014-03',
 'tranc_yearmonth_2014-04',
 'tranc_yearmonth_2014-05',
 'tranc_yearmonth_2014-06',
 'tranc_yearmonth_2014-07',
 'tranc_yearmonth_2014-08',
 'tranc_yearmonth_2014-09',
 

In [64]:
# Check binary columns
model_3_numerical_columns

['2room_sold',
 '3room_sold',
 'floor_area_sqft',
 'hawker_food_stalls',
 'vacancy',
 'hdb_age^2',
 'hdb_age max_floor_lvl',
 'hdb_age 2room_sold',
 'hdb_age 4room_sold',
 'hdb_age 1room_rental',
 'hdb_age mid_storey',
 'hdb_age floor_area_sqft',
 'hdb_age bus_stop_nearest_distance',
 'hdb_age mrt_nearest_distance',
 'hdb_age mall_nearest_distance',
 'hdb_age mall_within_1km',
 'hdb_age mall_within_2km',
 'hdb_age hawker_food_stalls',
 'hdb_age hawker_market_stalls',
 'hdb_age hawker_nearest_distance',
 'hdb_age hawker_within_500m',
 'hdb_age vacancy',
 'hdb_age pri_sch_nearest_distance',
 'hdb_age sec_sch_nearest_dist',
 'hdb_age 1room_sold',
 'max_floor_lvl 4room_sold',
 'max_floor_lvl 5room_sold',
 'max_floor_lvl floor_area_sqft',
 'max_floor_lvl bus_stop_nearest_distance',
 'max_floor_lvl mall_nearest_distance',
 'max_floor_lvl mall_within_500m',
 'max_floor_lvl mall_within_1km',
 'max_floor_lvl mall_within_2km',
 'max_floor_lvl hawker_food_stalls',
 'max_floor_lvl hawker_market_st

In [65]:
# Drop target variable 'resale_price'from numerical columns
model_3_numerical_columns.remove('resale_price')

# check numerical columns
model_3_numerical_columns

['2room_sold',
 '3room_sold',
 'floor_area_sqft',
 'hawker_food_stalls',
 'vacancy',
 'hdb_age^2',
 'hdb_age max_floor_lvl',
 'hdb_age 2room_sold',
 'hdb_age 4room_sold',
 'hdb_age 1room_rental',
 'hdb_age mid_storey',
 'hdb_age floor_area_sqft',
 'hdb_age bus_stop_nearest_distance',
 'hdb_age mrt_nearest_distance',
 'hdb_age mall_nearest_distance',
 'hdb_age mall_within_1km',
 'hdb_age mall_within_2km',
 'hdb_age hawker_food_stalls',
 'hdb_age hawker_market_stalls',
 'hdb_age hawker_nearest_distance',
 'hdb_age hawker_within_500m',
 'hdb_age vacancy',
 'hdb_age pri_sch_nearest_distance',
 'hdb_age sec_sch_nearest_dist',
 'hdb_age 1room_sold',
 'max_floor_lvl 4room_sold',
 'max_floor_lvl 5room_sold',
 'max_floor_lvl floor_area_sqft',
 'max_floor_lvl bus_stop_nearest_distance',
 'max_floor_lvl mall_nearest_distance',
 'max_floor_lvl mall_within_500m',
 'max_floor_lvl mall_within_1km',
 'max_floor_lvl mall_within_2km',
 'max_floor_lvl hawker_food_stalls',
 'max_floor_lvl hawker_market_st

In [66]:
# Check total count of columns
len(model_3_binary_columns + model_3_numerical_columns)

502

In [67]:
# Separate binary and numerical columns
model_3_binary_data = model_3[model_3_binary_columns]
model_3_numerical_data = model_3[model_3_numerical_columns]

### c. Scale numerical data

In [68]:
# Standardize the numerical features using StandardScaler
ss3 = StandardScaler()
model_3_numerical_data_scaled = ss3.fit_transform(model_3_numerical_data)

### d. Creating Features (X) and Target Variable (y)

In [69]:
# Concatenate scaled numerical columns with binary columns
model_3_combined_data = pd.concat([pd.DataFrame(model_3_numerical_data_scaled, columns=model_3_numerical_columns), model_3_binary_data], axis=1)

In [70]:
# Separate the target variable 'resale_price' (y) and features (X) from model_3
y3 = model_3['resale_price']
X3 = model_3_combined_data

In [71]:
# Split the dataset into training and testing sets
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.2, random_state=42)

### e. Model Development

In [72]:
# Instantiate a Linear Regression model
lr3 = LinearRegression()

In [73]:
# Instantiate a Lasso Regression model
lasso3 = LassoCV(n_alphas = 200, max_iter=100000)

In [74]:
# Instantiate a Ridge Regression model
r_alphas = np.logspace(-2, 2, 100)
ridge3 = RidgeCV(alphas = r_alphas)

In [75]:
# Calculate cross-validated scores for Linear Regression
lr_scores3 = cross_val_score(lr3, X_train3, y_train3, cv=5)

# Printing the scores
print("Cross-validated R2 scores for Linear Regression model:")
print(lr_scores3)
print("Mean R2 score:", np.mean(lr_scores3))

Cross-validated R2 scores for Linear Regression model:
[ 9.19167423e-01  9.21496113e-01  9.22270595e-01  9.17250233e-01
 -1.17662920e+17]
Mean R2 score: -2.3532584084620044e+16


In [76]:
# Cross-validation for Lasso Regression
lasso_scores3 = cross_val_score(lasso3, X_train3, y_train3, cv=5)

# Printing the scores
print("Cross-validated R2 scores for Lasso Regression model:")
print(lasso_scores3)
print("Mean R2 score:", np.mean(lasso_scores3))

Cross-validated R2 scores for Lasso Regression model:
[0.91391378 0.91371766 0.91433625 0.90886551 0.91314398]
Mean R2 score: 0.912795434714053


In [77]:
# Cross-validation for Ridge Regression
ridge_scores3 = cross_val_score(ridge3, X_train3, y_train3, cv=5)

# Printing the scores
print("Cross-validated R2 scores for Ridge Regression model:")
print(ridge_scores3)
print("Mean R2 score:", np.mean(ridge_scores3))

Cross-validated R2 scores for Ridge Regression model:
[0.92104989 0.92148697 0.92225971 0.91722988 0.92234135]
Mean R2 score: 0.9208735583866476


In [78]:
# Fit the Lasso model to the training data
lasso3.fit(X_train3, y_train3)

LassoCV(max_iter=100000, n_alphas=200)

In [79]:
# Calculate R2 scores
train_r2_lasso3 = lasso3.score(X_train3, y_train3)
test_r2_lasso3 = lasso3.score(X_test3, y_test3)

# Calculate RMSE
train_rmse_lasso3 = np.sqrt(metrics.mean_squared_error(y_train3, lasso3.predict(X_train3)))
test_rmse_lasso3 = np.sqrt(metrics.mean_squared_error(y_test3, lasso3.predict(X_test3)))

# Print the results
print("Lasso Model - Training R2 score:", train_r2_lasso3)
print("Lasso Model - Testing R2 score:", test_r2_lasso3)
print("Lasso Model - Training RMSE:", train_rmse_lasso3)
print("Lasso Model - Testing RMSE:", test_rmse_lasso3)

Lasso Model - Training R2 score: 0.9133547906737991
Lasso Model - Testing R2 score: 0.9116587093938212
Lasso Model - Training RMSE: 42170.44484923427
Lasso Model - Testing RMSE: 42503.85657090398


In [80]:
# Fit the Ridge model to the training data
ridge3.fit(X_train3, y_train3)

RidgeCV(alphas=array([1.00000000e-02, 1.09749877e-02, 1.20450354e-02, 1.32194115e-02,
       1.45082878e-02, 1.59228279e-02, 1.74752840e-02, 1.91791026e-02,
       2.10490414e-02, 2.31012970e-02, 2.53536449e-02, 2.78255940e-02,
       3.05385551e-02, 3.35160265e-02, 3.67837977e-02, 4.03701726e-02,
       4.43062146e-02, 4.86260158e-02, 5.33669923e-02, 5.85702082e-02,
       6.42807312e-02, 7.05480231e-0...
       1.17681195e+01, 1.29154967e+01, 1.41747416e+01, 1.55567614e+01,
       1.70735265e+01, 1.87381742e+01, 2.05651231e+01, 2.25701972e+01,
       2.47707636e+01, 2.71858824e+01, 2.98364724e+01, 3.27454916e+01,
       3.59381366e+01, 3.94420606e+01, 4.32876128e+01, 4.75081016e+01,
       5.21400829e+01, 5.72236766e+01, 6.28029144e+01, 6.89261210e+01,
       7.56463328e+01, 8.30217568e+01, 9.11162756e+01, 1.00000000e+02]))

In [81]:
# Calculate R2 scores
train_r2_ridge3 = ridge3.score(X_train3, y_train3)
test_r2_ridge3 = ridge3.score(X_test3, y_test3)

# Calculate RMSE
train_rmse_ridge3 = np.sqrt(metrics.mean_squared_error(y_train3, ridge3.predict(X_train3)))
test_rmse_ridge3 = np.sqrt(metrics.mean_squared_error(y_test3, ridge3.predict(X_test3)))

# Print the results
print("Ridge Model - Training R2 score:", train_r2_ridge3)
print("Ridge Model - Testing R2 score:", test_r2_ridge3)
print("Ridge Model - Training RMSE:", train_rmse_ridge3)
print("Ridge Model - Testing RMSE:", test_rmse_ridge3)

Ridge Model - Training R2 score: 0.9217340548543091
Ridge Model - Testing R2 score: 0.9207540375707537
Ridge Model - Training RMSE: 40079.50247499751
Ridge Model - Testing RMSE: 40256.40972282406


In [82]:
coefficients3 = ridge3.coef_

# Create a new DataFrame with predictors and coefficients
df_ridge3 = pd.DataFrame({"predictors": X3.columns, "coefficients": coefficients3})

# Sort the DataFrame ordered from largest to smallest coefficient by absolute values
df_ridge_sorted3 = df_ridge3.loc[df_ridge3["coefficients"].abs().sort_values(ascending=False).index]

# Reset the index
df_ridge_sorted3.reset_index(drop=True, inplace=True)

# Display all rows:
pd.set_option('display.max_rows', None)

#sorted dataframes
print("\nModel 3 Coefficients:")
print(df_ridge_sorted3)


Model 3 Coefficients:
                                            predictors   coefficients
0                                   flat_model_Terrace  221700.524458
1                                planning_area_Tanglin  220088.740682
2                            planning_area_Bukit Timah  180129.573907
3                          planning_area_Marine Parade  176823.487115
4                        full_flat_type_3 ROOM Terrace  147712.551261
5                              planning_area_Sembawang -117724.170014
6                              planning_area_Woodlands -108902.810163
7                                 planning_area_Outram  108545.775960
8                          planning_area_Bukit Panjang  -82529.390686
9                          planning_area_Choa Chu Kang  -79612.819159
10                                       market_hawker  -79594.976275
11                                planning_area_Bishan   77720.082598
12                            planning_area_Queenstown   75973.3113

## 05. Model 4 - Polynomial Features (Adjusted for Multicollinearity and polynomial features that do not contribute meaningfully to the prediction of the targer variable)

### a. Load Data

In [83]:
# Load data:
model_4 = pd.read_csv('./output/model_4.csv')

# Check data:
print(model_4.shape)
model_4.head()

(149805, 340)


,Unnamed: 0,2room_sold,3room_sold,floor_area_sqft,hdb_age^2,hdb_age 2room_sold,hdb_age 4room_sold,hdb_age bus_stop_nearest_distance,hdb_age mrt_nearest_distance,hdb_age mall_nearest_distance,hdb_age mall_within_2km,hdb_age hawker_food_stalls,hdb_age hawker_market_stalls,hdb_age hawker_nearest_distance,hdb_age hawker_within_500m,hdb_age vacancy,hdb_age pri_sch_nearest_distance,hdb_age sec_sch_nearest_dist,max_floor_lvl 4room_sold,max_floor_lvl 5room_sold,max_floor_lvl floor_area_sqft,max_floor_lvl bus_stop_nearest_distance,max_floor_lvl mall_nearest_distance,max_floor_lvl mall_within_500m,max_floor_lvl mall_within_1km,max_floor_lvl mall_within_2km,max_floor_lvl hawker_food_stalls,max_floor_lvl hawker_market_stalls,max_floor_lvl hawker_within_500m,max_floor_lvl hawker_within_2km,max_floor_lvl vacancy,max_floor_lvl pri_sch_nearest_distance,max_floor_lvl sec_sch_nearest_dist,total_dwelling_units floor_area_sqft,total_dwelling_units mrt_nearest_distance,total_dwelling_units mall_nearest_distance,total_dwelling_units hawker_food_stalls,total_dwelling_units cutoff_point,2room_sold 3room_sold,2room_sold mall_within_2km,3room_sold 4room_sold,3room_sold exec_sold,3room_sold mall_within_500m,3room_sold mall_within_1km,3room_sold mall_within_2km,3room_sold hawker_nearest_distance,3room_sold hawker_within_500m,4room_sold 5room_sold,4room_sold mall_within_1km,4room_sold mall_within_2km,4room_sold hawker_market_stalls,4room_sold hawker_within_500m,4room_sold hawker_within_2km,4room_sold sec_sch_nearest_dist,5room_sold bus_stop_nearest_distance,5room_sold mrt_nearest_distance,5room_sold mall_nearest_distance,5room_sold mall_within_500m,5room_sold mall_within_1km,5room_sold hawker_food_stalls,5room_sold hawker_market_stalls,5room_sold hawker_within_500m,5room_sold hawker_within_2km,5room_sold pri_sch_nearest_distance,5room_sold sec_sch_nearest_dist,exec_sold floor_area_sqft,exec_sold mall_within_500m,exec_sold mall_within_1km,exec_sold hawker_market_stalls,exec_sold hawker_nearest_distance,exec_sold hawker_within_500m,exec_sold hawker_within_2km,mid_storey floor_area_sqft,mid_storey bus_stop_nearest_distance,mid_storey mrt_nearest_distance,mid_storey mall_nearest_distance,mid_storey hawker_food_stalls,mid_storey pri_sch_nearest_distance,mid_storey sec_sch_nearest_dist,floor_area_sqft bus_stop_nearest_distance,floor_area_sqft mrt_nearest_distance,floor_area_sqft mall_nearest_distance,floor_area_sqft mall_within_1km,floor_area_sqft mall_within_2km,floor_area_sqft hawker_food_stalls,floor_area_sqft hawker_market_stalls,floor_area_sqft vacancy,floor_area_sqft pri_sch_nearest_distance,floor_area_sqft sec_sch_nearest_dist,bus_stop_nearest_distance hawker_within_2km,mrt_nearest_distance mall_nearest_distance,mrt_nearest_distance hawker_food_stalls,mrt_nearest_distance vacancy,mall_within_500m hawker_within_2km,mall_within_500m sec_sch_nearest_dist,mall_within_1km hawker_within_2km,mall_within_1km sec_sch_nearest_dist,mall_within_2km hawker_within_2km,mall_within_2km sec_sch_nearest_dist,hawker_food_stalls hawker_within_2km,hawker_market_stalls hawker_within_2km,hawker_nearest_distance hawker_within_2km,hawker_within_2km pri_sch_nearest_distance,hawker_within_2km sec_sch_nearest_dist,resale_price,commercial,market_hawker,multistorey_carpark,precinct_pavilion,bus_interchange,mrt_interchange,pri_sch_affiliation,affiliation,tranc_yearmonth_2012-04,tranc_yearmonth_2012-05,tranc_yearmonth_2012-06,tranc_yearmonth_2012-07,tranc_yearmonth_2012-08,tranc_yearmonth_2012-09,tranc_yearmonth_2012-10,tranc_yearmonth_2012-11,tranc_yearmonth_2012-12,tranc_yearmonth_2013-01,tranc_yearmonth_2013-02,tranc_yearmonth_2013-03,tranc_yearmonth_2013-04,tranc_yearmonth_2013-05,tranc_yearmonth_2013-06,tranc_yearmonth_2013-07,tranc_yearmonth_2013-08,tranc_yearmonth_2013-09,tranc_yearmonth_2013-10,tranc_yearmonth_2013-11,tranc_yearmonth_2013-12,tranc_yearmonth_2014-01,tranc_yearmonth_2014-02,tranc_yearmonth_2014-03,tranc_yearmonth_2014-04,tranc_yearmonth_2014-05,tranc_yearmonth_20

In [84]:
# Drop the 'Unnamed: 0' column from Model_4
model_4.drop(columns = ['Unnamed: 0'], inplace = True)

### b. Separate Numerical and Categorical Features

In [85]:
# Identify binary categorical columns (binary categorical columns have no changes across all our models)
model_4_binary_columns = model_1_binary_columns.copy()

model_4_binary_columns

['commercial',
 'market_hawker',
 'multistorey_carpark',
 'precinct_pavilion',
 'bus_interchange',
 'mrt_interchange',
 'pri_sch_affiliation',
 'affiliation',
 'tranc_yearmonth_2012-04',
 'tranc_yearmonth_2012-05',
 'tranc_yearmonth_2012-06',
 'tranc_yearmonth_2012-07',
 'tranc_yearmonth_2012-08',
 'tranc_yearmonth_2012-09',
 'tranc_yearmonth_2012-10',
 'tranc_yearmonth_2012-11',
 'tranc_yearmonth_2012-12',
 'tranc_yearmonth_2013-01',
 'tranc_yearmonth_2013-02',
 'tranc_yearmonth_2013-03',
 'tranc_yearmonth_2013-04',
 'tranc_yearmonth_2013-05',
 'tranc_yearmonth_2013-06',
 'tranc_yearmonth_2013-07',
 'tranc_yearmonth_2013-08',
 'tranc_yearmonth_2013-09',
 'tranc_yearmonth_2013-10',
 'tranc_yearmonth_2013-11',
 'tranc_yearmonth_2013-12',
 'tranc_yearmonth_2014-01',
 'tranc_yearmonth_2014-02',
 'tranc_yearmonth_2014-03',
 'tranc_yearmonth_2014-04',
 'tranc_yearmonth_2014-05',
 'tranc_yearmonth_2014-06',
 'tranc_yearmonth_2014-07',
 'tranc_yearmonth_2014-08',
 'tranc_yearmonth_2014-09',
 

In [86]:
# Identify numerical columns
model_4_numerical_columns = model_4.columns.copy()
model_4_numerical_columns = [i for i in model_4_numerical_columns if i not in model_4_binary_columns]

In [87]:
# Check binary columns 
model_4_binary_columns

['commercial',
 'market_hawker',
 'multistorey_carpark',
 'precinct_pavilion',
 'bus_interchange',
 'mrt_interchange',
 'pri_sch_affiliation',
 'affiliation',
 'tranc_yearmonth_2012-04',
 'tranc_yearmonth_2012-05',
 'tranc_yearmonth_2012-06',
 'tranc_yearmonth_2012-07',
 'tranc_yearmonth_2012-08',
 'tranc_yearmonth_2012-09',
 'tranc_yearmonth_2012-10',
 'tranc_yearmonth_2012-11',
 'tranc_yearmonth_2012-12',
 'tranc_yearmonth_2013-01',
 'tranc_yearmonth_2013-02',
 'tranc_yearmonth_2013-03',
 'tranc_yearmonth_2013-04',
 'tranc_yearmonth_2013-05',
 'tranc_yearmonth_2013-06',
 'tranc_yearmonth_2013-07',
 'tranc_yearmonth_2013-08',
 'tranc_yearmonth_2013-09',
 'tranc_yearmonth_2013-10',
 'tranc_yearmonth_2013-11',
 'tranc_yearmonth_2013-12',
 'tranc_yearmonth_2014-01',
 'tranc_yearmonth_2014-02',
 'tranc_yearmonth_2014-03',
 'tranc_yearmonth_2014-04',
 'tranc_yearmonth_2014-05',
 'tranc_yearmonth_2014-06',
 'tranc_yearmonth_2014-07',
 'tranc_yearmonth_2014-08',
 'tranc_yearmonth_2014-09',
 

In [88]:
# Check numerical columns 
model_4_numerical_columns

['2room_sold',
 '3room_sold',
 'floor_area_sqft',
 'hdb_age^2',
 'hdb_age 2room_sold',
 'hdb_age 4room_sold',
 'hdb_age bus_stop_nearest_distance',
 'hdb_age mrt_nearest_distance',
 'hdb_age mall_nearest_distance',
 'hdb_age mall_within_2km',
 'hdb_age hawker_food_stalls',
 'hdb_age hawker_market_stalls',
 'hdb_age hawker_nearest_distance',
 'hdb_age hawker_within_500m',
 'hdb_age vacancy',
 'hdb_age pri_sch_nearest_distance',
 'hdb_age sec_sch_nearest_dist',
 'max_floor_lvl 4room_sold',
 'max_floor_lvl 5room_sold',
 'max_floor_lvl floor_area_sqft',
 'max_floor_lvl bus_stop_nearest_distance',
 'max_floor_lvl mall_nearest_distance',
 'max_floor_lvl mall_within_500m',
 'max_floor_lvl mall_within_1km',
 'max_floor_lvl mall_within_2km',
 'max_floor_lvl hawker_food_stalls',
 'max_floor_lvl hawker_market_stalls',
 'max_floor_lvl hawker_within_500m',
 'max_floor_lvl hawker_within_2km',
 'max_floor_lvl vacancy',
 'max_floor_lvl pri_sch_nearest_distance',
 'max_floor_lvl sec_sch_nearest_dist',


In [89]:
# Drop target variable 'resale_price'from numerical columns
model_4_numerical_columns.remove('resale_price')

# check numerical columns
model_4_numerical_columns

['2room_sold',
 '3room_sold',
 'floor_area_sqft',
 'hdb_age^2',
 'hdb_age 2room_sold',
 'hdb_age 4room_sold',
 'hdb_age bus_stop_nearest_distance',
 'hdb_age mrt_nearest_distance',
 'hdb_age mall_nearest_distance',
 'hdb_age mall_within_2km',
 'hdb_age hawker_food_stalls',
 'hdb_age hawker_market_stalls',
 'hdb_age hawker_nearest_distance',
 'hdb_age hawker_within_500m',
 'hdb_age vacancy',
 'hdb_age pri_sch_nearest_distance',
 'hdb_age sec_sch_nearest_dist',
 'max_floor_lvl 4room_sold',
 'max_floor_lvl 5room_sold',
 'max_floor_lvl floor_area_sqft',
 'max_floor_lvl bus_stop_nearest_distance',
 'max_floor_lvl mall_nearest_distance',
 'max_floor_lvl mall_within_500m',
 'max_floor_lvl mall_within_1km',
 'max_floor_lvl mall_within_2km',
 'max_floor_lvl hawker_food_stalls',
 'max_floor_lvl hawker_market_stalls',
 'max_floor_lvl hawker_within_500m',
 'max_floor_lvl hawker_within_2km',
 'max_floor_lvl vacancy',
 'max_floor_lvl pri_sch_nearest_distance',
 'max_floor_lvl sec_sch_nearest_dist',


In [90]:
# Check total count of columns
len(model_4_binary_columns + model_4_numerical_columns)

338

In [91]:
# Separate binary and numerical columns
model_4_binary_data = model_4[model_4_binary_columns]
model_4_numerical_data = model_4[model_4_numerical_columns]

### c. Scale numerical data

In [92]:
# Standardize the numerical features using StandardScaler
ss4 = StandardScaler()
model_4_numerical_data_scaled = ss4.fit_transform(model_4_numerical_data)

### d. Creating Features (X) and Target Variable (y)

In [93]:
# Concatenate scaled numerical columns with binary columns
model_4_combined_data = pd.concat([pd.DataFrame(model_4_numerical_data_scaled, columns=model_4_numerical_columns), model_4_binary_data], axis=1)

In [94]:
# Separate the target variable 'resale_price' (y) and features (X) from model_4
y4 = model_4['resale_price']
X4 = model_4_combined_data

In [95]:
# Split the dataset into training and testing sets
X_train4, X_test4, y_train4, y_test4 = train_test_split(X4, y4, test_size=0.2, random_state=42)

### e. Model Development

In [96]:
# Instantiate a Linear Regression model
lr4 = LinearRegression()

In [97]:
# Instantiate a Lasso Regression model
lasso4 = LassoCV(n_alphas = 200, max_iter=100000)

In [98]:
# Instantiate a Ridge Regression model
r_alphas = np.logspace(-2, 2, 100)
ridge4 = RidgeCV(alphas = r_alphas)

In [99]:
# Calculate cross-validated scores for Linear Regression
lr_scores4 = cross_val_score(lr4, X_train4, y_train4, cv=5)

# Printing the scores
print("Cross-validated R2 scores for Linear Regression model:")
print(lr_scores4)
print("Mean R2 score:", np.mean(lr_scores4))

Cross-validated R2 scores for Linear Regression model:
[ 9.15635472e-01  9.15863847e-01  9.17247343e-01  9.11701391e-01
 -3.10504201e+17]
Mean R2 score: -6.210084010548072e+16


In [100]:
# Cross-validation for Lasso Regression
lasso_scores4 = cross_val_score(lasso4, X_train4, y_train4, cv=5)

# Printing the scores
print("Cross-validated R2 scores for Lasso Regression model:")
print(lasso_scores4)
print("Mean R2 score:", np.mean(lasso_scores4))

Cross-validated R2 scores for Lasso Regression model:
[0.90977529 0.90946794 0.91084644 0.90489505 0.90897589]
Mean R2 score: 0.9087921211536625


In [101]:
# Cross-validation for Ridge Regression
ridge_scores4 = cross_val_score(ridge4, X_train4, y_train4, cv=5)

# Printing the scores
print("Cross-validated R2 scores for Ridge Regression model:")
print(ridge_scores4)
print("Mean R2 score:", np.mean(ridge_scores4))

Cross-validated R2 scores for Ridge Regression model:
[0.91563629 0.91587657 0.91724477 0.91174673 0.9166518 ]
Mean R2 score: 0.9154312311483386


In [102]:
# Fit the Lasso model to the training data
lasso4.fit(X_train4, y_train4)

LassoCV(max_iter=100000, n_alphas=200)

In [103]:
# Calculate R2 scores
train_r2_lasso4 = lasso4.score(X_train4, y_train4)
test_r2_lasso4 = lasso4.score(X_test4, y_test4)

# Calculate RMSE
train_rmse_lasso4 = np.sqrt(metrics.mean_squared_error(y_train4, lasso4.predict(X_train4)))
test_rmse_lasso4 = np.sqrt(metrics.mean_squared_error(y_test4, lasso4.predict(X_test4)))

# Print the results
print("Lasso Model - Training R2 score:", train_r2_lasso4)
print("Lasso Model - Testing R2 score:", test_r2_lasso4)
print("Lasso Model - Training RMSE:", train_rmse_lasso4)
print("Lasso Model - Testing RMSE:", test_rmse_lasso4)

Lasso Model - Training R2 score: 0.9091988647647072
Lasso Model - Testing R2 score: 0.9070762083604914
Lasso Model - Training RMSE: 43169.94982130037
Lasso Model - Testing RMSE: 43592.31455334257


In [104]:
# Fit the Ridge model to the training data
ridge4.fit(X_train4, y_train4)

RidgeCV(alphas=array([1.00000000e-02, 1.09749877e-02, 1.20450354e-02, 1.32194115e-02,
       1.45082878e-02, 1.59228279e-02, 1.74752840e-02, 1.91791026e-02,
       2.10490414e-02, 2.31012970e-02, 2.53536449e-02, 2.78255940e-02,
       3.05385551e-02, 3.35160265e-02, 3.67837977e-02, 4.03701726e-02,
       4.43062146e-02, 4.86260158e-02, 5.33669923e-02, 5.85702082e-02,
       6.42807312e-02, 7.05480231e-0...
       1.17681195e+01, 1.29154967e+01, 1.41747416e+01, 1.55567614e+01,
       1.70735265e+01, 1.87381742e+01, 2.05651231e+01, 2.25701972e+01,
       2.47707636e+01, 2.71858824e+01, 2.98364724e+01, 3.27454916e+01,
       3.59381366e+01, 3.94420606e+01, 4.32876128e+01, 4.75081016e+01,
       5.21400829e+01, 5.72236766e+01, 6.28029144e+01, 6.89261210e+01,
       7.56463328e+01, 8.30217568e+01, 9.11162756e+01, 1.00000000e+02]))

In [105]:
# Calculate R2 scores
train_r2_ridge4 = ridge4.score(X_train4, y_train4)
test_r2_ridge4 = ridge4.score(X_test4, y_test4)

# Calculate RMSE
train_rmse_ridge4 = np.sqrt(metrics.mean_squared_error(y_train4, ridge4.predict(X_train4)))
test_rmse_ridge4 = np.sqrt(metrics.mean_squared_error(y_test4, ridge4.predict(X_test4)))

# Print the results
print("Ridge Model - Training R2 score:", train_r2_ridge4)
print("Ridge Model - Testing R2 score:", test_r2_ridge4)
print("Ridge Model - Training RMSE:", train_rmse_ridge4)
print("Ridge Model - Testing RMSE:", test_rmse_ridge4)

Ridge Model - Training R2 score: 0.916082903388791
Ridge Model - Testing R2 score: 0.9145497118151704
Ridge Model - Training RMSE: 41501.245396855855
Ridge Model - Testing RMSE: 41802.59412100827


In [106]:
coefficients4 = ridge4.coef_

# Create a new DataFrame with predictors and coefficients
df_ridge4 = pd.DataFrame({"predictors": X4.columns, "coefficients": coefficients4})

# Sort the DataFrame ordered from largest to smallest coefficient by absolute values
df_ridge_sorted4 = df_ridge4.loc[df_ridge4["coefficients"].abs().sort_values(ascending=False).index]

# Reset the index
df_ridge_sorted4.reset_index(drop=True, inplace=True)

# Display all rows:
pd.set_option('display.max_rows', None)

#sorted dataframes
print("\nModel 4 Coefficients:")
print(df_ridge_sorted4)


Model 4 Coefficients:
                                           predictors   coefficients
0                                  flat_model_Terrace  225639.307326
1                               planning_area_Tanglin  219484.274418
2                         planning_area_Marine Parade  204935.987383
3                           planning_area_Bukit Timah  180623.190771
4                       full_flat_type_3 ROOM Terrace  152093.275073
5                             planning_area_Sembawang -109123.202624
6                             planning_area_Woodlands  -95589.847565
7                                planning_area_Changi  -92866.181834
8                                planning_area_Bishan   84870.307236
9                                 flat_model_Improved  -83971.501460
10                           planning_area_Queenstown   82679.790561
11                        planning_area_Downtown Core   81233.594492
12                      full_flat_type_4 ROOM Terrace   73546.032253
13         

## 06. Evaluate results of all regression models

| Model | Method | Training R2 Score | Testing R2 Score | Training RMSE | Testing RMSE |
|-------|--------|-------------------|------------------|---------------|--------------|
| 1     | Lasso  | 0.8963            | 0.8932           | 46142.23      | 46728.65     |
| 1     | Ridge  | 0.9026            | 0.9004           | 44721.66      | 45140.20     |
| 2     | Lasso  | 0.9221            | 0.9202           | 39973.46      | 40388.08     |
| 2     | Ridge  | 0.9330            | 0.9320           | 37089.04      | 37301.21     |
| 3     | Lasso  | 0.9134            | 0.9117           | 42170.44      | 42503.86     |
| 3     | Ridge  | 0.9217            | 0.9208           | 40079.50      | 40256.41     |
| 4     | Lasso  | 0.9092            | 0.9071           | 43169.95      | 43592.31     |
| 4     | Ridge  | 0.9161            | 0.9145           | 41501.24      | 41802.59     |


- Based on the R2 scores and RMSE values, Model_2 with Ridge regularization appears to be the best model. It has the highest testing R2 score (0.9320), indicating that it explains the most variance in the target variable. Additionally, it has the lowest testing RMSE (37301.20), suggesting that its predictions are, on average, closest to the actual values. However, it’s important to consider the complexity of the model and ensure it’s not overfitting the training data.

- However, in Ridge Regression 2, multicollinearity (a situation where two or more features are highly correlated) could lead to the following issues:
  - Unstable Coefficients: The model might assign an unreasonably high positive weight to one feature and a high negative weight to another, even though they are conveying similar information.
  - Difficulty in Interpretation: It becomes difficult to interpret the coefficients of the features, as a change in one feature is tangled up with changes in the other correlated features.
  - Overfitting: The model might overfit to the noise in the training data, leading to poor performance on unseen data.

- Hence, by addressing multicollinearity in Ridge Regression 3, these issues are mitigated. Moreover, Model 3 demonstrates only a marginal decrease in performance compared to Model 2, establishing it as a superior model in terms of stability and interpretability.

- Comparison of Model 3 and 4: We have noted from the results that Model 3 ridge has outperformed Model 4. Regularization methods like Ridge and Lasso are capable of feature selection by shrinking the coefficients of less important features towards zero. This automated feature selection can often be more effective and less biased than manual feature removal based on correlation with the target variable. Therefore, Ridge Regression 3, which allows the regularization to perform feature selection, may be a better choice.

In conclusion, we have determined Ridge Regression 3 to be our best model, given its balance of performance, interpretability, and feature selection.